# Hashtag Grouping on Twitter

Input: 
    Hastag scrap result from cekmedsos_database.vw_ttidata

Output: 
    Mapping tables to group each hashtag entry with similarity
    

What we need to do....
1. Load the table entry from mySQL into python
2. Read the entry

Pre-requisite
pip install wheel
pip install pandas

### 1. Create Connection to mySQL

ref to this page:
    [How to Use Python with mySQL in Jupyter](https://medium.com/@tattwei46/how-to-use-python-with-mysql-79304bee8753)
    
first, we need to install mysql connector

!pip install psycopg2

In [2]:
import pymysql.cursors
import pandas as pd
import uuid
import time
import os
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd


### Input as proces started
Record header and parameter information

 * Kesepakatan status di kolom screen_analisis_ai.status
 * 1 --> baru diinput
 * 2 --> lagi dikerjakan
 * 3 --> proses berhasil
 * 4 --> proses gagal
 *
 
 * Kesepakatan jenis analisa AI
 * 1 --> Analisa Cluster
 * 2 --> Analisa image clustering
 * 3 --> Analisa sentiment
 
 

In [3]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query_psql(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [4]:
from datetime import datetime

# get available jobs from database server, first come first serve
sql = "select id, hastag, parameter \
from screen_analisis_ai \
where active = '1' \
and status = '1' \
and jenis_analisa = '1' \
order by created asc, id asc limit 1"

# print(sql)
row_count = execute_query_psql(sql)

if( len(row_count) ) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()

# row_count.head()

database_keyword_id = row_count['hastag'][0]
similarity_treshold = 0.9
i_process_id = row_count['id'][0]
screen_name = ''

print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)

87270
0.9
3044


### Marking this process as running
so that another process will not take this one


In [ ]:
# Prepare SQL Statement
print(i_process_id)
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = %s"
sql = sql.replace('%s', str(i_process_id))

print(sql)
row_count = execute_query_psql(sql)
print('update ' + str(row_count) + ' rows')


### Runnning Process


In [ ]:
#
# Create Header Record
sql = "insert into ret_analysis_header (job_id, datetime_start, user_id) values (" + str(i_process_id) + ", now(), '1')"
# Execute the query
print(sql)
row_count = execute_query_psql(sql)


In [ ]:
#
# Create Parameter Record
sql = """
insert into ret_analysis_parameter 
(job_id, param_id, param_name, param_value) 
values (%s, %s, %s, %s)
""" % (str(i_process_id), '1', "'Similarity Treshold'", str(similarity_treshold))
# Execute the query
print(sql)
row_count = execute_query_psql(sql)

sql = """
insert into ret_analysis_parameter 
(job_id, param_id, param_name, param_value) 
values (%s, %s, %s, %s)
""" % (str(i_process_id), '1', "'DB_ID'", str(database_keyword_id))
print(sql)
row_count = execute_query_psql(sql)


#### Starting process, 
Run query against RDBMS


In [ ]:
#
# Query to get tweet data, apply analitics to this dataset
#
s_query_string = 'select id, tweet, tweet_date_time from ret_tweet where '
# s_query_string = 'select id, tweet, tweet_date_time from vw_ret_tweet_clean where '

if (screen_name != ''):
    # print('use screen name')
    s_query_string = s_query_string + 'screen_name = "' + screen_name + '" and db_id = "' + str(database_keyword_id) + '"'
else:
    # print('no use')
    s_query_string = s_query_string + "db_id = '" + str(database_keyword_id).replace("'","") + "'"
    
print(s_query_string)
df = execute_query_psql(s_query_string)

# Close Connection

# see result
df.head()

### 2. Try to pre-process all the text

target-> tokenizing into another dataframe

In [ ]:
import string
import re
import nltk

from nltk.tokenize import word_tokenize

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    
    return strOut
# end Text Preprocessing


# Apply to data frame
df['tweet'] = df['tweet'].apply(text_preproc)

df.head()

### Finish preprocessing, Tokenized
Next step, is to output to new column for tokenized sentence

In [ ]:
from nltk.corpus import stopwords 

# Load stopwords
sw = stopwords.words('indonesian')

# apply tokenize
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

# apply stopword removal
df['sw'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

df.head()

### Define function to calculate similarity
Function return similarity score

In [ ]:
def calculate_similarity(X_set, Y_set):
# Program to measure the similarity between  
# two sentences using cosine similarity. 

    l1 =[];l2 =[]

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i]
    try:
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
    except:
        # print('zero div on X_set')
        return 0;
        
    return cosine

def largest_in_col(arr,nCol):
    # 
    # Find largest value of col nCol on 2D arr
    #
    
    # init value
    max_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] > max_val:
            max_val = arr[x][nCol]
            row_index = x
        
    return max_val,row_index

def smallest_in_col(arr,nCol):
    # 
    # Find smallest value of col nCol on 2D arr
    #
    
    # init value
    min_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] < min_val:
            min_val = arr[x][nCol]
            row_index = x
        
    return min_val,row_index


### Try to using function
using some of array cells, create n matrices

1. take one tweet, compare to all data set
2. flag 1 if similar
3. take next tweet, if similar from prev tweet, skip
4. if not similar, add counter, then proceed

In [ ]:
import numpy as np
import array as arr
from tqdm import tqdm

# set to max CPU Cores
# multitasking.set_max_threads(multitasking.config["CPU_CORES"] * 5)

st = similarity_treshold
cluster_no = 1
s_score = 0
s_score_current = 0
i_current_cluster = 0

#create zero element array
#col 0 => index base tweet
#col 1 => cluster number
#col 2 => similarity score
base_tweet = []

#proceed to compare to all tweet
cosine_matrix = np.zeros(( len(df), len(df) ), dtype=np.dtype('f4'))

# flag the mt
with tqdm(total=( (len(df)*len(df)/2))-(len(df)/2))  as pbar:
    for j in range(0, len(df)):
        tweet_to_compare = df['sw'][j]

        #check, is this second tweet?
        if(j == 0):
            #first tweet, add as cluster no #1
            base_tweet.append([j,1,1.0])
            i_current_cluster = base_tweet[0][1]

        elif(j == 1):
            #compare to prev tweet
            s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[0][0] ])

            if(s_score < st):
                #not similar
                base_tweet.append([j,2,1])
                i_current_cluster = base_tweet[j][1]

        else:
            #other else tweet
            for x in range(0,len(base_tweet)):
                #compare every element
                s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[x][0] ])
                base_tweet[x][2] = s_score

            if(largest_in_col(base_tweet,2)[0] < st):
                #no similar, add as one new cluster
                i_current_cluster = i_current_cluster + 1
                base_tweet.append([j,i_current_cluster,largest_in_col(base_tweet,2)[0]])
            else:
                #determine cluster# from biggest similarity
                i_current_cluster = base_tweet[(largest_in_col(base_tweet,2)[1])][1]

        #proceed to compare to all tweet
        for i in range(0,len(df)):
            # update progress
            if (j<i):
                pbar.update(1)
                s_score = calculate_similarity(tweet_to_compare, df['sw'][i])
                if (s_score >= st):
                    cosine_matrix[i,j] = i_current_cluster


pbar.close()
print(cosine_matrix)

### Writing result to file
Using NPZ format for efficiency, and try to load them after save


In [ ]:
# save numpy array as npz file
from numpy import asarray
from numpy import savez_compressed

# save to npy file
# savez_compressed('./output/df_380.npz',df)
# savez_compressed('./output/data_380.npz', cosine_matrix)
# savez_compressed('./output/data_380_base.npz', base_tweet)

print(len(cosine_matrix))
print('Save data complete .... ')

### How to save to rdbms?
save the header data, meta data of the process
[ret_analysis_header]
- JobID
- user initiated
- Time Started
- Time End

Process Parameter
[ret_analysis_parameter]
- JobID
- Param Name
- Param Value
    - ST Value
    - DB ID
    - Screen Name
    - How Many Tweet analyzed

save detail cluster information
[ret_base_tweet]
- JobID
- Tweet Base# --> tweet ID
- Cluster#

save detail calculation result, structure
[ret_cluster_result]
- JobID
- TweetID
- Cluster

In [ ]:
#
# Create Base Tweet Record
sql = "insert into ret_base_tweet (job_id, tweet_id, cluster_id) values (%s, %s, %s)"

## inserting base tweet
for i in range(0,len(base_tweet)):
    # Execute the query
    execute_query_psql(sql % (i_process_id, df['id'][i], base_tweet[i][1]) )
    

print('finished inserting base tweet record')

### Record cluster result
into table ret_cluster_result

In [ ]:
from numpy import asarray
from numpy import savetxt

# savetxt('data.csv', cosine_matrix, delimiter=',')

In [ ]:
#
# Create Tweet Cluster Record
sql = "insert into ret_cluster_result (job_id, tweet_id, cluster_no) values (%s, %s, %s)"

print(len(cosine_matrix))
print(i_process_id)

# initiate cluster number
i_cluster_no_save = 0
temp_val = 0
                 
## inserting tweet cluster
for i in range(0, len(cosine_matrix)):
    
    # find value in this particular row
    for j in range(0,len(cosine_matrix[i])):
        
        # print(cosine_matrix[i][j])
        temp_val = cosine_matrix[j][i]
        
        if(temp_val != 0):
            i_cluster_no_save = temp_val

    # Execute the query
    execute_query_psql(sql % (i_process_id, df['id'][i], i_cluster_no_save))

print('finished inserting cluster data')

### Record finish time
update table ret_analysis_header

In [ ]:
# Calculate result
stored_procedure_query = "CALL spInsertResultToMV(%s);"
execute_query_psql( stored_procedure_query % (i_process_id) )
#

In [ ]:

#
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
execute_query_psql(sql % (i_process_id, 1, "'#Tweet Processed'",len(df)))

#
# Create Tweet Cluster Record
sql = "update ret_analysis_header set datetime_finish = NOW() where job_id = %s"
# Executing query
execute_query_psql(sql % (i_process_id) )

print(i_process_id)
print('job finished')

### Marking as finished the job
on this particular i_process_id

In [ ]:
sql = "update screen_analisis_ai set status = 3, duration = EXTRACT(EPOCH FROM (now() - start_process)), end_process = NOW() where id = %s"
execute_query_psql(sql % (i_process_id))

print('inserting result finished')



### Done Here ...
1. Create data structure to record below things
    Job Header
    - JobID
    - Similarity Treshold
    - Dataset Parameter
        - By User
        - By Keyword
    - Running time start
    - Running time end
    
    Dataset Result
        - base_tweet
        - cosine_matrix
        
2. Push the result data into RDBMS Server, in this case is mySQL
    - upload csv file to mySQL
    - import to database
  
Entry on crontab
* * * * * /home/haviz/ai-project/tweet-grouping/run_one.sh >> /home/haviz/ai-project/tweet_gorup.log 2>&1

In [ ]:
# Wait 10 sec before release
time.sleep(10)